In [ ]:
!pip install -U ultralytics

In [ ]:
import os
import sys
from collections import deque
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import imageio # Video IO
from collections import deque


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")
if DEVICE == "cuda":
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("WARNING: GPU not detected. YOLO will run on CPU.")

Using device: cuda
GPU Name: NVIDIA GeForce RTX 5050 Laptop GPU


In [ ]:
from ultralytics import YOLO
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# Load YOLOv8 Nano (fast, sufficient for people detection)
yolo_model = YOLO("yolov8n.pt")
yolo_model.to(DEVICE)

print("YOLO loaded successfully")

Using device: cuda
YOLO loaded successfully


In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

CUDA available: True
GPU: NVIDIA GeForce RTX 5050 Laptop GPU


In [ ]:
# Video I/O
VIDEO_PATH = "/home/naman/Task-0/volleyball_match.mp4"
OUTPUT_PATH = "/home/naman/Task-0/volleyball_output.mp4"

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError("Cannot open video")

FPS = int(cap.get(cv2.CAP_PROP_FPS))
WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(
    OUTPUT_PATH,
    cv2.VideoWriter_fourcc(*"mp4v"),
    FPS,
    (WIDTH, HEIGHT)
)

MID_X = WIDTH // 2

In [ ]:
# Ball HSV config
LOWER_YELLOW = np.array([20, 100, 100])
UPPER_YELLOW = np.array([35, 255, 255])

LOWER_BLUE = np.array([90, 80, 50])
UPPER_BLUE = np.array([130, 255, 255])

KERNEL = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

# Ball detection
def detect_ball(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    mask_y = cv2.inRange(hsv, LOWER_YELLOW, UPPER_YELLOW)
    mask_b = cv2.inRange(hsv, LOWER_BLUE, UPPER_BLUE)
    mask = cv2.bitwise_or(mask_y, mask_b)

    mask = cv2.GaussianBlur(mask, (5,5), 0)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, KERNEL)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, KERNEL)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    best = None
    max_area = 0

    for c in contours:
        area = cv2.contourArea(c)
        if area < 50:
            continue
        peri = cv2.arcLength(c, True)
        if peri == 0:
            continue
        circ = 4 * np.pi * area / (peri * peri)
        if circ > 0.6 and area > max_area:
            (x, y), r = cv2.minEnclosingCircle(c)
            best = (int(x), int(y), int(r))
            max_area = area

    return best

In [ ]:
# Kalman filter (ball)

kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1,0,0,0],[0,1,0,0]], np.float32)
kalman.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03

ball_trail = deque(maxlen=64)


# Court mask

def apply_court_mask(frame):
    h, w = frame.shape[:2]
    poly = np.array([
        [150, int(0.35*h)],
        [w-150, int(0.35*h)],
        [w-80, h-80],
        [80, h-80]
    ], np.int32)

    mask = np.zeros((h,w), np.uint8)
    cv2.fillPoly(mask, [poly], 255)
    return cv2.bitwise_and(frame, frame, mask=mask)

In [ ]:
# Player detection (YOLO)

def detect_players(frame):
    results = yolo_model(
        frame,
        conf=0.4,
        classes=[0],
        device=DEVICE,
        verbose=False
    )

    dets = []
    for r in results:
        if r.boxes is None:
            continue
        for b in r.boxes:
            x1,y1,x2,y2 = b.xyxy[0].cpu().numpy()
            dets.append([int(x1),int(y1),int(x2),int(y2)])
    return dets

In [ ]:
# Simple SORT tracker

class Track:
    def __init__(self, bbox, tid):
        self.bbox = bbox
        self.id = tid
        self.miss = 0

class Tracker:
    def __init__(self):
        self.tracks = []
        self.next_id = 0

    def iou(self, a, b):
        xA,yA = max(a[0],b[0]), max(a[1],b[1])
        xB,yB = min(a[2],b[2]), min(a[3],b[3])
        inter = max(0,xB-xA)*max(0,yB-yA)
        areaA = (a[2]-a[0])*(a[3]-a[1])
        areaB = (b[2]-b[0])*(b[3]-b[1])
        return inter / (areaA+areaB-inter+1e-6)

    def update(self, dets):
        updated = []
        for d in dets:
            matched = False
            for t in self.tracks:
                if self.iou(d, t.bbox) > 0.3:
                    t.bbox = d
                    t.miss = 0
                    updated.append(t)
                    matched = True
                    break
            if not matched:
                updated.append(Track(d, self.next_id))
                self.next_id += 1

        for t in self.tracks:
            if t not in updated:
                t.miss += 1
                if t.miss < 10:
                    updated.append(t)

        self.tracks = updated
        return self.tracks

tracker = Tracker()

In [ ]:
# Team classification

def classify_team(frame, bbox):
    x1,y1,x2,y2 = bbox
    torso = frame[y1:y1+(y2-y1)//2, x1:x2]
    if torso.size == 0:
        return None
    hsv = cv2.cvtColor(torso, cv2.COLOR_BGR2HSV)
    h = np.mean(hsv[:,:,0])
    if 15 < h < 40 or 90 < h < 130:
        return "A"
    if h < 10 or h > 160:
        return "B"
    return None

In [ ]:
# Main loop

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Ball
    ball = detect_ball(frame)
    if ball:
        x,y,r = ball
        kalman.correct(np.array([[x],[y]], np.float32))
        ball_trail.append((x,y))
        cv2.circle(frame, (x,y), r, (0,255,255), 2)
    else:
        pred = kalman.predict()
        ball_trail.append((int(pred[0]), int(pred[1])))

    for i in range(1,len(ball_trail)):
        cv2.line(frame, ball_trail[i-1], ball_trail[i], (0,255,255), 2)q

    # Players
    masked = apply_court_mask(frame)
    dets = detect_players(masked)
    tracks = tracker.update(dets)

    counts = {"A_L":0,"A_R":0,"B_L":0,"B_R":0}

    for t in tracks:
        team = classify_team(frame, t.bbox)
        if team is None:
            continue
        x1,y1,x2,y2 = t.bbox
        cx = (x1+x2)//2
        side = "L" if cx < MID_X else "R"
        counts[f"{team}_{side}"] += 1

        color = (0,255,0) if team=="A" else (0,0,255)
        cv2.rectangle(frame,(x1,y1),(x2,y2),color,2)
        cv2.putText(frame,f"ID {t.id} T{team}",(x1,y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)

    cv2.putText(frame,f"A Left:{counts['A_L']} Right:{counts['A_R']}",
                (20,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)
    cv2.putText(frame,f"B Left:{counts['B_L']} Right:{counts['B_R']}",
                (20,60),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)

    out.write(frame)
    cv2.imshow("Volleyball Analysis", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

/tmp/ipykernel_14658/2951316556.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ball_trail.append((int(pred[0]), int(pred[1])))


KeyboardInterrupt: 

: 